###  怎样更好地更深，更大？
- 提问:AlexNet比LeNet更深更大，怎样更好地更深，更大？
- 选项：
1. 更多的全连接层（太贵：大+计算量大）
2. 更多的卷积层（在AlexNet的基础上加）
3. 将卷积层组合成块->VGG!

## VGG的idea
- 实现了VGG块
- 实验表明：同样计算开销的情况下，对更多的3 * 3 比堆少一点的5 * 5效果要好
- VGG块： 3 * 3卷积（pad = 1）(n层，m通道), 2 * 2最大池化层（stride = 2）
![softmax-description](./imgs/25-1.png)

## VGG的架构
- 多个VGG块后连接全连接层
- VGG可以理解为更大更深的VGG
![softmax-description](./imgs/25-3.png)

## 总结
- VGG使用可重复的卷积快来构建深度卷积神经网络
- 使用不同卷积块个数和超参数可以得到不同复杂度的变种
- VGG-11 使用可复用的卷积块构造网络。不同的 VGG 模型可通过每个块中卷积层数量和输出通道数量的差异来定义。块的使用导致网络定义的非常简洁。使用块可以有效地设计复杂的网络。
- 在VGG论文中，Simonyan和Ziserman尝试了各种架构。特别是他们发现深层且窄的卷积（即3×3）比较浅层且宽的卷积更有效。

### 卷积输出层的尺寸公式
假设：输入图片（Input）大小为I * I，卷积核（Filter）大小为K * K，步长（stride）为S，填充（Padding）的像素数为P，那卷积层输出（Output）的特征图大小为多少呢?

可以得出推导公式：

$O=((I-K+2P)/S)+1$

### 最大池化层的输出尺寸
$outwidth = （inwidth+2 * pad-kernelsize）/stride+1$

In [1]:
%matplotlib inline
import torch
from torch import nn
import d2l

In [2]:
# 构造vgg块
# 先创建空列表，然后把layer放进列表，最后放到nn.Sequential里面去
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        # 这里输入和输出的尺寸是相同的 (i-3+2)+1
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        
        layers.append(nn.ReLU())
        # 在num_convs中，就第一个卷积层改变通道数，后续卷积层都不改变通道数
        in_channels = out_channels 
    # 通过最大池化层来减小feature map的长宽
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers) 

In [3]:
# 构造vgg网络
# 224/2^5 = 7，7就除不动了，所以是5块
# 经典的思想：把网络分成5块，每一块高宽减半（在maxpooling时去减半），通道数翻倍
# （num_convs, out_channels）
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [4]:
# VGG11的结构
def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    # 下一层的in_channels就是上一层的out_channels
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, # 卷积块的组合
        nn.Flatten(),
        # 全连接层部分
        # 7是自己计算得到的
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))
net = vgg(conv_arch)

In [ ]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

/Users/huhan/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])


In [2]:
# 训练模型
# 因为vgg11比AlexNet计算量更大，我们构建了一个通道数较少的网络
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

NameError: name 'conv_arch' is not defined

In [ ]:
# lr会增大一些（随便调的）
lr, num_epochs, batch_size = 0.05, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())